In [16]:
import os
import sys
import mmap

sys.path.append("/Users/zhd/packages/CaImAn/")
sys.path.append("/Users/zhd/packages/develop/voltagedenoising/")
sys.path.append("/Users/zhd/Desktop/Research/Project/Dendritic data/code/")
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
#import scipy as sp
import superpixel_analysis as sup
import filter_data as fdata
import caiman as cm
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
data_dir = "/Users/zhd/Desktop/Research/Project/MIP data/data/"

In [11]:
out_dir = "/Users/zhd/Desktop/Research/Project/MIP data/result/try4/"

In [4]:
import h5py
with h5py.File(data_dir + "MIP_xy_and_yz_mouse17_run3_1_forliam.mat", 'r') as f:
    print(list(f.keys()))
    MIP_yz =f['MIP_yz'][()];
    MIP_xy =f['MIP_xy'][()];

['#refs#', 'MIP_xy', 'MIP_yz', 'info', 'processing']


In [5]:
MIP_yz = MIP_yz.transpose([1,2,0])
MIP_xy = MIP_xy.transpose([1,2,0])

In [7]:
from skimage.external.tifffile import imsave
imsave('MIP_xy_564_140_-750.tif', cm.movie(MIP_xy.transpose([2,0,1])))

In [12]:
fname_short= 'MIP_xy_564_140_-750.tif'
fname =[fname_short]
#cm.movie(MIP_xy.transpose([2,0,1])).save(fname_short)
#fname_new = cm.save_memmap([os.path.split(fname_short)[1]], base_name='Yr1', is_3D=False)
# check directory
#fname_short= 'Yr1_d1_160_d2_160_d3_1_order_F_frames_5000_.mmap'

# Motion correction

In [9]:
fname =[fname_short]
m_orig = cm.load_movie_chain(fname)
downsample_ratio = .2  # motion can be perceived better when downsampling in time
offset_mov = 0;#np.min(m_orig);  # if the data has very negative values compute an offset value

100%|██████████| 1/1 [00:00<00:00,  2.35it/s]


In [10]:
niter_rig = 1        # number of iterations for rigid motion correction
max_shifts = (2, 2)  # maximum allowed rigid shift in pixels (view the movie to get a sense of motion)
splits_rig =  10     # for parallelization split the movies in  num_splits chuncks across time
                     # if none all the splits are processed and the movie is saved
#num_splits_to_process_rig = None  # intervals at which patches are laid out for motion correction
strides =  (24, 24)  # create a new patch every x pixels for pw-rigid correction
overlaps = (12, 12)  # overlap between pathes (size of patch strides+overlaps)
splits_els =  10     # for parallelization split the movies in  num_splits chuncks across time
#num_splits_to_process_els = [28, None]
upsample_factor_grid = 4  # upsample factor to avoid smearing when merging patches
max_deviation_rigid = 3   # maximum deviation allowed for patch with respect to rigid shifts

In [11]:
dview=None

In [12]:
# create a motion correction object
from caiman.motion_correction import MotionCorrect, tile_and_correct, motion_correction_piecewise

mc = MotionCorrect(fname[0], offset_mov,
                   dview = dview, max_shifts = max_shifts, niter_rig = niter_rig,
                   splits_rig = splits_rig, 
                   strides = strides, overlaps= overlaps, splits_els = splits_els,
                   upsample_factor_grid = upsample_factor_grid,
                   max_deviation_rigid = max_deviation_rigid, 
                   shifts_opencv = True, nonneg_movie = True)

In [ ]:
mc.motion_correct_rigid(save_movie=True)
# load motion corrected movie
m_rig = cm.load(mc.fname_tot_rig)
bord_px_rig = np.ceil(np.max(mc.shifts_rig)).astype(np.int)
#%% visualize templates
plt.figure(figsize = (20,10))
plt.imshow(mc.total_template_rig, cmap = 'gray')

In [ ]:
#%% plot rigid shifts
plt.close()
plt.figure(figsize = (20,10))
plt.plot(mc.shifts_rig)
plt.legend(['x shifts','y shifts'])
plt.xlabel('frames')
plt.ylabel('pixels')

In [ ]:
#%%capture
#%% motion correct piecewise rigid
mc.motion_correct_pwrigid(save_movie=True, template=mc.total_template_rig)
m_els = cm.load(mc.fname_tot_els)
m_els.resize(1, 1, downsample_ratio).play(
    gain=1, offset = 0, fr=30, magnification=1,bord_px = bord_px_rig)

In [ ]:
#%% visualize elastic shifts
plt.close()
plt.figure(figsize = (20,10))
plt.subplot(2, 1, 1)
plt.plot(mc.x_shifts_els)
plt.ylabel('x shifts (pixels)')
plt.subplot(2, 1, 2)
plt.plot(mc.y_shifts_els)
plt.ylabel('y_shifts (pixels)')
plt.xlabel('frames')
#%% compute borders to exclude
bord_px_els = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                 np.max(np.abs(mc.y_shifts_els)))).astype(np.int)

## save MC results

In [114]:
fname[0][:-4]

'MIP_xy_564_140_-750'

In [130]:
np.save(data_dir + fname[0][:-4]+'_rig.npy',np.array(m_rig.transpose(1,2,0)))

In [131]:
m_rig.shape

(564, 140, 750)

In [132]:
np.save(data_dir + fname[0][:-4]+'_els.npy',np.array(m_els.transpose(1,2,0)))

In [133]:
m_els.shape

(564, 140, 750)

# High pass filter and Wiener filter

In [5]:
m_rig = np.load(data_dir + fname[0][:-4]+'_rig.npy')

In [ ]:
## check trend ##
plt.plot(m_rig[20,300,:])

In [7]:
dims = m_rig.shape
m_rig_r= m_rig.reshape((np.prod(dims[:2]),dims[2]),order='F')

# get rid of trend
if True:
    fs = 10000.0
    cutoff = 150
    order = 1
    Y_detr = tfd.butter_highpass(m_rig_r, cutoff, fs, order)
    offset_1= m_rig_r-Y_detr
    Y_detr = Y_detr.reshape(dims,order="F")

In [ ]:
plt.plot(offset_1[:6,:].T);

In [ ]:
plt.plot(Y_detr[:,:,:].mean(axis=0).mean(axis=0))

In [ ]:
plt.plot(Y_detr[20,350,:])

In [194]:
# Caserati representation d x T
dims = m_rig.shape
m_rig_r= m_rig.reshape((np.prod(dims[:2]),dims[2]),order='F')

# get rid of trend
if False:
    Y_detr, offset = tools.remove_trend(m_rig_r);
    Y_detr = Y_detr.reshape(dims,order="F")

In [ ]:
plt.plot(m_rig[120,30,:])

In [11]:
########## normalize each pixel by the estimated noise
mean_ = Y_detr.mean(axis=2,keepdims=True);
sn_ = sup.noise_estimator(Y_detr);
sn_ = sn_.reshape(dims[0],dims[1],1)
Y_norm = (Y_detr - mean_)/sn_;

In [ ]:
Y_wf, _ = fdata.spatial_filter_image(Y_norm);

## plot results and make movies

In [664]:
Cno = sup.local_correlations_fft(m_rig);
Cnh = sup.local_correlations_fft(Y_norm);
Cnw = sup.local_correlations_fft(Y_wf);

In [ ]:
########### check correlation image ######################
fig = plt.figure(figsize=(48,24))
ax1 = plt.subplot(3,1,1)
sup.show_img(ax1, Cno);
ax1.set(title="Local mean correlation for raw data")
ax1.title.set_fontsize(15)
ax1.title.set_fontweight("bold")
ax2 = plt.subplot(3,1,2)
sup.show_img(ax2, Cnh);
ax2.set(title="Local mean correlation for HPF data")
ax2.title.set_fontsize(15)
ax2.title.set_fontweight("bold")
ax3 = plt.subplot(3,1,3)
sup.show_img(ax3, Cnw);
ax3.set(title="Local mean correlation for wiener filter data")
ax3.title.set_fontsize(15)
ax3.title.set_fontweight("bold")
plt.tight_layout();
fig.savefig(out_dir + "raw_HPF_WF_corr_compare.pdf")

In [680]:
################ compare movie #########################
raw_1 = m_orig.T;#([2,0,1]);30000
rig_1 = m_rig.T;#30000
hpf_1 = Y_norm.T;
els_1 = Y_wf1.T;
raw_1 = raw_1 - raw_1.min(axis=0, keepdims=True);
rig_1 = rig_1 - rig_1.min(axis=0, keepdims=True);
els_1 = els_1 - els_1.min(axis=0, keepdims=True);
hpf_1 = hpf_1 - hpf_1.min(axis=0, keepdims=True);
raw_1 =(raw_1 - raw_1.min())/(raw_1.max() - raw_1.min())*255;
rig_1 =(rig_1 - rig_1.min())/(rig_1.max() - rig_1.min())*255;
els_1 =(els_1 - els_1.min())/(els_1.max() - els_1.min())*255;
hpf_1 =(hpf_1 - hpf_1.min())/(hpf_1.max() - hpf_1.min())*255;

In [ ]:
plt.hist(raw_1.ravel());
raw_1 = np.clip(raw_1,0,25);

In [ ]:
plt.hist(rig_1.ravel());
rig_1 = np.clip(rig_1,0,25);

In [ ]:
plt.hist(els_1.ravel());
els_1 = np.clip(els_1,0,25);

In [ ]:
plt.hist(hpf_1.ravel());
hpf_1 = np.clip(hpf_1,0,50);

In [685]:
raw_1 =(raw_1 - raw_1.min())/(raw_1.max() - raw_1.min())*255;
rig_1 =(rig_1 - rig_1.min())/(rig_1.max() - rig_1.min())*255;
els_1 =(els_1 - els_1.min())/(els_1.max() - els_1.min())*255;
hpf_1 =(hpf_1 - hpf_1.min())/(hpf_1.max() - hpf_1.min())*255;

In [686]:
Frame=np.concatenate([raw_1,rig_1,hpf_1,els_1],axis=2);

In [687]:
import skvideo.io
outputdata = Frame
writer = skvideo.io.FFmpegWriter(out_dir + "raw_rig_hpf_els.mp4",inputdict={'-r': '20'})
for i in range(564):
    writer.writeFrame(outputdata[i, :, :])
writer.close()

In [ ]:
import matplotlib.animation as animation

fig = plt.figure(figsize=(12,16));
ax1 = plt.subplot(1,4,1)
#ax1 = fig.add_subplot(111)
ax1.set(title="raw data")
ax1.title.set_fontsize(15)
ax1.title.set_fontweight("bold")
ax1.set_xticks([])
ax1.set_yticks([])
ax2 = plt.subplot(1,4,2)
ax2.set(title="MC")
ax2.title.set_fontsize(15)
ax2.title.set_fontweight("bold")
ax2.set_xticks([])
ax2.set_yticks([])
ax3 = plt.subplot(1,4,3)
#ax1 = fig.add_subplot(111)
ax3.set(title="HPF")
ax3.title.set_fontsize(15)
ax3.title.set_fontweight("bold")
ax3.set_xticks([])
ax3.set_yticks([])
ax4 = plt.subplot(1,4,4)
ax4.set(title="WF")
ax4.title.set_fontsize(15)
ax4.title.set_fontweight("bold")
ax4.set_xticks([])
ax4.set_yticks([])
plt.tight_layout()

img1 = ax1.imshow(raw_1[0,:,:],vmin=raw_1.min(),vmax=raw_1.max(),cmap="Greys_r");
img2 = ax2.imshow(rig_1[0,:,:],vmin=rig_1.min(),vmax=rig_1.max(),cmap="Greys_r");
img3 = ax3.imshow(hpf_1[0,:,:],vmin=hpf_1.min(),vmax=hpf_1.max(),cmap="Greys_r");    
img4 = ax4.imshow(els_1[0,:,:],vmin=els_1.min(),vmax=els_1.max(),cmap="Greys_r");    

def update(i):
    img1.set_data(raw_1[i+1,:,:]);
    img2.set_data(rig_1[i+1,:,:])
    img3.set_data(hpf_1[i+1,:,:])
    img4.set_data(els_1[i+1,:,:])
    return [img1,img2,img3,img4]

ani = animation.FuncAnimation(fig, update, frames=564-1, blit=True)
ani.save(out_dir + 'raw_mc_hpf_wf_label.mp4', fps=20, metadata={'artist':'zhd'})

In [13]:
Y_wf1 = Y_wf*sn_ + mean_;
Y_wf1 = Y_wf1 - Y_wf1.min(axis=2,keepdims=True)

In [15]:
np.save(data_dir + fname[0][:-4]+ "_Y_wf1.npy", Y_wf1)

# superpixel analysis

In [13]:
Y_wf1 = np.load(data_dir + fname[0][:-4]+ "_Y_wf1.npy") ## data has to be non-negative

In [14]:
Y_wf1 = Y_wf1[:,30:-30,:] ## cut wiener filtered data because in this region there's no neurons

In [ ]:
start=time.time()
rlt2=sup.axon_pipeline(Y_wf1, cut_off_point=[0.9,0.8], length_cut=[15,10], th=[2,1], pass_num=2, 
                    residual_cut = [0.6,0.6], corr_th_fix=0.4, max_allow_neuron_size=0.3, 
                    merge_corr_thr=0.6, merge_overlap_thr=0.6, num_plane=1, patch_size=[100,100],
                    plot_en=False, TF=True, fudge_factor=1, text=True, bg=False,
                    max_iter=35, max_iter_fin=65, update_after=4)
print(time.time()-start)

## plot results

In [28]:
dims = Y_wf1.shape

In [99]:
rltk = rlt2["fin_rlt"].copy()

In [104]:
rltk["corr_img_all_r"].shape

(140, 690, 302)

In [51]:
Ys = (np.matmul(rltk["a"], rltk["c"].T) + rltk["b"]).reshape(dims,order="F");

In [52]:
Ys_tf = (np.matmul(rltk["a"], rltk["c_tf"].T) + rltk["b"]).reshape(dims,order="F");
Ys = (np.matmul(rltk["a"], rltk["c"].T) + rltk["b"]).reshape(dims,order="F");

In [72]:
Yo_1 = Y_wf1.T#transpose([2,0,1]);
Ys_1 = Ys.T#transpose([2,0,1]);
Ys_tf_1 = Ys_tf.T#transpose([2,0,1]);
Rt_1 = Yo_1 - Ys_1;#Yd_1 - Yy_1; ## residual 
Yo_1 = Yo_1 - Yo_1.min(axis=0, keepdims=True);
Ys_tf_1 = Ys_tf_1 - Ys_tf_1.min(axis=0, keepdims=True);
Rt_1 = Rt_1 - Rt_1.min(axis=0, keepdims=True);

In [73]:
scale1=(Ys_tf_1.max() - Ys_tf_1.min())/(Yo_1.max() - Yo_1.min())
scale1

0.9211309432029817

In [74]:
scale2=(Rt_1.max() - Rt_1.min())/(Yo_1.max() - Yo_1.min())
scale2

0.6592055190183381

In [75]:
Ys_tf_1 = (Ys_tf_1 - Ys_tf_1.min())/(Ys_tf_1.max() - Ys_tf_1.min())*255*scale1;
Rt_1 =(Rt_1 - Rt_1.min())/(Rt_1.max() - Rt_1.min())*255*scale2;
Yo_1 = (Yo_1 - Yo_1.min())/(Yo_1.max() - Yo_1.min())*255;

In [ ]:
plt.hist(Yo_1.ravel())
Yo_1 = np.clip(Yo_1,0,25)

In [ ]:
plt.hist(Ys_tf_1.ravel())
Ys_tf_1 = np.clip(Ys_tf_1,0,25)

In [ ]:
plt.hist(Rt_1.ravel())
#Rt_1 = np.clip(Rt_1,0,29)
Rt_1 = np.clip(Rt_1,0,16)

In [79]:
Ys_tf_1 = (Ys_tf_1 - Ys_tf_1.min())/(Ys_tf_1.max() - Ys_tf_1.min())*255*scale1;
Rt_1 =(Rt_1 - Rt_1.min())/(Rt_1.max() - Rt_1.min())*255*scale2;
Yo_1 = (Yo_1 - Yo_1.min())/(Yo_1.max() - Yo_1.min())*255;

In [80]:
Frame=np.concatenate([Yo_1,Ys_tf_1,Rt_1],axis=2);

In [81]:
import skvideo.io
outputdata = Frame
writer = skvideo.io.FFmpegWriter(out_dir + "wf_sig_res-2pass.mp4",inputdict={'-r': '20'})
for i in range(dims[2]):
    writer.writeFrame(outputdata[i, :, :])
writer.close()

In [ ]:
import matplotlib.animation as animation

fig = plt.figure(figsize=(9,15));
ax1 = plt.subplot(1,3,1)
#ax1 = fig.add_subplot(111)
ax1.set(title="HPF + WF data")
ax1.title.set_fontsize(15)
ax1.title.set_fontweight("bold")
ax1.set_xticks([])
ax1.set_yticks([])
ax2 = plt.subplot(1,3,2)
ax2.set(title="Signal")
ax2.title.set_fontsize(15)
ax2.title.set_fontweight("bold")
ax2.set_xticks([])
ax2.set_yticks([])
ax3 = plt.subplot(1,3,3)
#ax1 = fig.add_subplot(111)
ax3.set(title="Residual")
ax3.title.set_fontsize(15)
ax3.title.set_fontweight("bold")
ax3.set_xticks([])
ax3.set_yticks([])
plt.tight_layout()

img1 = ax1.imshow(Yo_1[0,:,:],vmin=Yo_1.min(),vmax=Yo_1.max(),cmap="Greys_r");
img2 = ax2.imshow(Ys_tf_1[0,:,:],vmin=Ys_tf_1.min(),vmax=Ys_tf_1.min() + ((Ys_tf_1.max()-Ys_tf_1.min())/scale1),cmap="Greys_r");
img3 = ax3.imshow(Rt_1[0,:,:],vmin=Rt_1.min(),vmax=Rt_1.min() + ((Rt_1.max()-Rt_1.min())/scale2),cmap="Greys_r");    

def update(i):
    img1.set_data(Yo_1[i+1,:,:]);
    img2.set_data(Ys_tf_1[i+1,:,:])
    img3.set_data(Rt_1[i+1,:,:])
    return [img1,img2,img3]

ani = animation.FuncAnimation(fig, update, frames=dims[2]-1, blit=True)
ani.save(out_dir + 'wf_sig_res_label-2pass.mp4', fps=20, metadata={'artist':'zhd'})

In [88]:
fig = sup.spatial_comp_plot(rltk["a"][:,200:], rltk["corr_img_all_r"][:,:,200:]);
fig.savefig(out_dir + "spatial_Yd_2pass-3.pdf")
#fig.savefig(out_dir + "spatial_Yd_3pass_first-2.pdf")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [92]:
fig = sup.temporal_compare_plot(rltk["c"][:,:],rltk["c_tf"][:,:]);
#fig.savefig(out_dir + "temp_orig.pdf")
fig.savefig(out_dir + "temp_Yd_2pass.pdf")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
